# Simulating Gaussian Splats Using Simplicits
Lets simulate some gaussian splat objects using [Simplicits](https://research.nvidia.com/labs/toronto-ai/simplicits/)  within the Jupyter notebook

## Installation Requirements
When installing [INRIA's splatting and rasterization code](https://github.com/graphdeco-inria/gaussian-splatting), make sure the paths and packages are setup correctly since we will need both Kaolin as well as INRIA's modules.

For compatability with INRIA, we recommend using:
- pytorch 2.0.1
- cuda 11.8
- mkl 2024.0
- setuptools 69.5
- matplotlib
- plyfile

**Comment/uncomment below** to toggle installation of Inria's Gaussian Splatting into `examples/tutorial/physics/inria/`

In [ ]:
#### Setup and Installation ###

### Install necessary packages
!pip install -q plyfile
!pip install k3d

### Create an inria folder
%mkdir inria
%cd inria

### Clone the repo recursively
!git clone --recursive https://github.com/graphdeco-inria/gaussian-splatting.git

### Install the submodules
%cd gaussian-splatting
!pip install submodules/diff-gaussian-rasterization
!pip install submodules/simple-knn

### Check Location
INRIA's gaussian splatting is not a package, so in order to use it we need to clone the repository and work within it.

Make sure you're in the `..../kaolin/examples/tutorial/physics/inria/gaussian-splatting` directory. If not, `%cd` into it in order to be able to use their code.


In [ ]:
# %cd inria/gaussian-splatting

In [ ]:
# Gaussian splatting dependencies
from utils.graphics_utils import focal2fov
from utils.system_utils import searchForMaxIteration
from gaussian_renderer import render, GaussianModel
from scene.cameras import Camera as GSCamera
from utils.general_utils import strip_symmetric, build_scaling_rotation
%pwd

## Pre-trained Splats from AWS
Lets grab a few pre-trained gaussian splat models from AWS.
We can unzip and set the splat model path below to the correct `.ply` file.

In [ ]:
# Download and unzip the nerfsynthetic bulldozer
!wget https://nvidia-kaolin.s3.us-east-2.amazonaws.com/data/dozer.zip -P output/
!unzip output/dozer.zip -d output/

# # Download and unzip the nerfsynthetic hotdog
# !wget https://nvidia-kaolin.s3.us-east-2.amazonaws.com/data/dozer.zip -P output/
# !unzip output/dozer.zip -d output/

# # Download and unzip the nerfsynthetic ficus
# !wget https://nvidia-kaolin.s3.us-east-2.amazonaws.com/data/dozer.zip -P output/
# !unzip output/dozer.zip -d output/

model_path = 'output/dozer/point_cloud/iteration_30000/point_cloud.ply'

## Load and Display the Splats Model Using Kaolin
After the setup, we can use kaolin to load and display the splat model within the Jupyter notebook.

In [ ]:
import copy
import ipywidgets
import json
import kaolin

import matplotlib.pyplot as plt
import numpy as np
import os
import logging
import sys
import time
import threading  
from pathlib import Path
from functools import partial


import torch
import torchvision

from IPython.display import display
from ipywidgets import Button, HBox, VBox

logging.basicConfig(level=logging.INFO, stream=sys.stdout)
logger = logging.getLogger(__name__)

def log_tensor(t, name, **kwargs):
    print(kaolin.utils.testing.tensor_info(t, name=name, **kwargs))


In [ ]:
class PipelineParamsNoparse:
    """ Same as PipelineParams but without argument parser. """
    def __init__(self):
        self.convert_SHs_python = False
        self.compute_cov3D_python = True # covariances will be updated during simulation
        self.debug = False

def load_model(model_path, sh_degree=3, iteration=-1):
    # Load guassians
    gaussians = GaussianModel(sh_degree)
    gaussians.load_ply(model_path)                                                 
    return gaussians

gaussians = load_model(model_path)
pipeline = PipelineParamsNoparse()
background = torch.tensor([1, 1, 1], dtype=torch.float32, device="cuda") # Set white bg

## Render Using Kaolin Camera Conventions

In order to easily view splats in the notebook, let's set up Gaussian Splat rendering using Kaolin Camera conventions.

In [ ]:
# TODO: Use the new cam converter code.
def compute_cam_fov(intrinsics, axis='x'):
    # compute FOV from focal
    aspectScale = intrinsics.width / 2.0
    tanHalfAngle = aspectScale / (intrinsics.focal_x if axis == 'x' else intrinsics.focal_y).item()
    fov = np.arctan(tanHalfAngle) * 2
    return fov

def convert_kaolin_camera(kal_camera):
    """ Convert kaolin camera to GS camera. """
    R = kal_camera.extrinsics.R[0]
    R[1:3] = -R[1:3]
    T = kal_camera.extrinsics.t.squeeze()
    T[1:3] = -T[1:3]
    return GSCamera(colmap_id=0,
                    R=R.transpose(1, 0).cpu().numpy(), 
                    T=T.cpu().numpy(), 
                    FoVx=compute_cam_fov(kal_camera.intrinsics, 'x'), 
                    FoVy=compute_cam_fov(kal_camera.intrinsics, 'y'), 
                    image=torch.zeros((3, kal_camera.height, kal_camera.width)),  # fake 
                    gt_alpha_mask=None,
                    image_name='fake',
                    uid=0)

resolution=512
static_scene_kalcam = kaolin.render.easy_render.default_camera(resolution)

# Change coordinate system since INRIA's convention uses z-axis as the up axis.  
static_scene_kalcam.extrinsics.change_coordinate_system(torch.tensor([[1,0,0],[0,0,-1], [0,1,0]]))
static_scene_kalcam.intrinsics.zoom(-50)

def render_kaolin(kaolin_cam):
    cam = convert_kaolin_camera(kaolin_cam)
    render_res = render(cam, gaussians, pipeline, background)
    rendering = render_res["render"]
    return (torch.clamp(rendering.permute(1, 2, 0), 0, 1) * 255).to(torch.uint8).detach().cpu()

focus_at = (static_scene_kalcam.cam_pos() - 4. * static_scene_kalcam.extrinsics.cam_forward()).squeeze()
static_scene_viz = kaolin.visualize.IpyTurntableVisualizer(
    resolution, resolution, copy.deepcopy(static_scene_kalcam), render_kaolin, 
    focus_at=None, world_up_axis=2, max_fps=12)
static_scene_viz.show()

## Create a Simplicits Object and Train
[Simplicits](https://research.nvidia.com/labs/toronto-ai/simplicits/) is a mesh-free, representation-agnostic way to simulation elastic deformations. We can use it to simulate Gaussian Splats at interactive rates within the jupyter notebook.

Use the simplicits `easy_api` to create, train and simulate a simplicits object.

First lets set some material parameters

In [ ]:
# Physics material parameters. 
# Use some approximated values, or look them up online.
soft_youngs_modulus = 21000
poisson_ratio = 0.45
rho = 100  # kg/m^3
approx_volume = 3  # m^3

### Training
Next we create a `SimplicitsObject` and train its skinning weight functions.

In [ ]:
# Initialize and train a Simpicits object to enable simulation
# Points sampled over the object
pos = gaussians.get_xyz.clone().detach().cuda()       
yms = torch.full((pos.shape[0],), soft_youngs_modulus, device="cuda")
prs = torch.full((pos.shape[0],), poisson_ratio, device="cuda")
rhos = torch.full((pos.shape[0],), rho, device="cuda")

sim_obj = kaolin.physics.simplicits.SimplicitsObject(pos, yms, prs, rhos, torch.tensor([approx_volume], dtype=torch.float32, device="cuda"), num_samples=2048, model_layers=10, num_handles=40)
print('Training simplicits object. This will take 2-3min. ')
start = time.time()
sim_obj.train(num_steps=20000)
end = time.time()
print(f"Ends training in {end-start} seconds")


In [ ]:
# sim_obj.save_model("../dozer_model.pt")
# sim_obj.load_model("../dozer_model.pt")

## Setup Scene Using Simplicits Easy API
Lets create an empty scene with default parameters, then reset the max number of newton steps to 5 for faster runtimes.

In [ ]:
scene = kaolin.physics.simplicits.SimplicitsScene() # Create a default scene # default empty scene
scene.max_newton_steps = 3 #Convergence might not be guaranteed at few NM iterations, but runs very fast

Now we add our object to the scene. We use 2048 cubature points to integrate over.

In [ ]:
# The scene copies it into an internal SimulatableObject utility class
obj_idx = scene.add_object(sim_obj, num_cub_pts=2048)

Lets set set gravity and floor forces on the scene

In [ ]:
# Add gravity to the scene
scene.set_scene_gravity(acc_gravity=torch.tensor([0, 0, 9.8]))
# Add floor to the scene
scene.set_scene_floor(floor_height=-0.7, floor_axis=2, floor_penalty=1000, flip_floor=False)

We can play around with the material parameters of the object, indicated via object_idx

In [ ]:
# Make the object softer by updating the material parameter
scene.set_object_materials(obj_idx, yms=torch.tensor(10000, device='cuda', dtype=torch.float))
scene.set_object_materials(obj_idx, rhos=torch.tensor(100, device='cuda', dtype=torch.float))

Finally we can add a boundary condition to fix the bottom of the splats. Comment this cell out if you want to skip having a boundary condition.

In [ ]:
def boundary_func(pts):
    # Extract the z-coordinates (height) of the points
    heights = pts[:, 2]
    # Determine the minimum and maximum z-coordinates
    z_min = torch.min(heights)
    z_max = torch.max(heights)
    # Calculate the threshold z-coordinate for the bottom 5% of the object's height
    threshold = z_min + 0.08 * (z_max - z_min)
    # Get the indices of the points in the upper 10%
    return heights <= threshold

boundary = scene.set_object_boundary_condition(obj_idx, "boundary1", boundary_func, bdry_penalty=10000)

## Thats it! Now lets simulate the splats
As the splats deform, we must update their scale, rotation via the deformation gradient.

In [ ]:
def build_covariance_from_scaling_rotation_deformations(scaling, scaling_modifier, rotation, defo_grad=None):
    L = build_scaling_rotation(scaling_modifier * scaling, rotation)
    if defo_grad==None:
        FL = L
    else:
        FL = torch.bmm(defo_grad, L)
    actual_covariance = FL @ FL.transpose(1, 2)
    symm = strip_symmetric(actual_covariance)
    return symm

gaussians.covariance_activation = build_covariance_from_scaling_rotation_deformations

Next we display the simulation

In [ ]:
def fast_render(in_cam, factor=8):
    lowres_cam = copy.deepcopy(in_cam)
    lowres_cam.width = in_cam.width // factor
    lowres_cam.height = in_cam.height // factor
    return render(lowres_cam)

global sim_thread_open, sim_thread
sim_thread_open = False
sim_thread = None

def run_sim():
    for s in range(int(100)):
        with new_vis.out:
            scene.run_sim_step()
            print(".", end="")
            with torch.no_grad():
                gaussians._xyz = scene.get_object_deformed_pts(obj_idx, pos).squeeze()
                F = scene.get_object_deformation_gradient(obj_idx, pos).squeeze()
                build_cov = partial(build_covariance_from_scaling_rotation_deformations, defo_grad=F)
                gaussians.covariance_activation = build_cov
        new_vis.render_update()

def start_simulation(b):
    global sim_thread_open, sim_thread
    with new_vis.out:
        if(sim_thread_open):
            sim_thread.join()
            sim_thread_open = False
        sim_thread_open = True
        sim_thread = threading.Thread(target=run_sim, daemon=True)
        sim_thread.start()

scene.reset()
new_kal_cam = kaolin.render.easy_render.default_camera(resolution)
new_kal_cam.extrinsics.change_coordinate_system(torch.tensor([[1,0,0],[0,0,-1], [0,1,0]]))
new_kal_cam.intrinsics.zoom(-50)

button = Button(description='Run Sim')
button.on_click(start_simulation)

new_vis = kaolin.visualize.IpyTurntableVisualizer(
    512, 512, copy.deepcopy(new_kal_cam), render_kaolin, 
    focus_at=gaussians._xyz.mean().cpu(), world_up_axis=2, max_fps=6)
new_vis.render_update()
display(HBox([new_vis.canvas, button]), new_vis.out)